# Water Surface Elevation in 3D

In [ ]:
import xarray as xr

from schimpy import schism_mesh

import pandas as pd
import holoviews as hv
from holoviews import opts, dim

## Open output dataset containing the water surface elevations

In [ ]:
ds = xr.open_mfdataset('../tests/data/m1_hello_schism/outputs/out2d_*.nc', concat_dim='time', combine="nested",
                  data_vars='minimal', coords='minimal', compat='override')
ds

## Read mesh data of elements and nodes

This might not work for quad elements. Will have to deal with those as two non-overlapping triangles

In [ ]:
smesh = schism_mesh.read_mesh('../tests/data/m1_hello_schism/hgrid.gr3')

dfelems = pd.DataFrame(smesh.elems,columns=[0,1,2])
#dfelems

dfnodes = pd.DataFrame(smesh.nodes, columns=['x','y','z'])
dfnodes.z = -dfnodes.z
#dfnodes

## TriSurface plots from Plotly

Needed to add simplices from existing elements information of mesh rather than Delaunay triangulation of nodes

#Adapted from https://anaconda.org/philippjfr/brain/notebook?version=2017.05.04.1924

The code below allows for the simplices already defined by elems to be used instead of doing a Delaunay triangulation (used from scipy as a way to calculate the simplices)

In [ ]:
hv.extension('plotly')
from schism_viz import plotly_ext

dfelev = dfnodes.copy()

#ds.elevation.min(), ds.elevation.max()

mesh_surface = plotly_ext.TriSurface(dfelev, simplices = dfelems.values).opts(plot_edges=False, cmap='gray')
mesh_surface

## Show mesh and water surface

The water surface simplices are derived from the nodes with Delaunay triangulation. Can we get the simplices from the information in the schism output files?

In [ ]:
dfsurface = dfnodes.copy()

def show_surface(time=0):
    dfsurface.z = ds.elevation.values[time,:]
    water_surface = plotly_ext.TriSurface(dfsurface, simplices = dfelems.values).opts(width=800, zlim=(-10,2))
    return water_surface.opts(plot_edges=False, cmap='kbc', clim=(0,2), colorbar=True)

In [ ]:
show_surface(0)

## Now combine both to animate both bathymetry and water surface

In [ ]:
def show_combined(time):
    return show_surface(time)*mesh_surface

In [ ]:
import panel as pn
pn.extension()

In [ ]:
time_slider = pn.widgets.IntSlider(name='Time Index Slider', start=0, end=len(ds.time)-1)

In [ ]:
pn.Column(pn.pane.Markdown("""# Water Surface Animation in 3D
 * Move the time slider below to move the surface through time
 * Use mouse wheel zoom to zoom in 
 * Use mouse to rotate and view from different angles"""),
       time_slider, 
       hv.DynamicMap(show_combined, streams={'time':time_slider}).opts(
           width=800, height=800)).servable(title='Water Surface Elevation Animation in 3D')#.show()